# Eliminación por Bloques con el apoyo el método matricial de factorización QR 

## Introducción

Visualmente, la solución de sistemas de ecuaciones lineales (SEL) implica encontrar la intersección de rectas, planos o hiperplanos (dependiendo de las dimensiones en las que nos encontremos). Para la solución de este tipo de sistemas, existen diversos métodos, que son escogidos de conformidad con las características del sistema objeto de resolución (en específico, sus coeficientes).

Un forma de resolver estos SEL consiste en la eliminación de un subconjunto de variables y resolver un sistema más pequeño (`eliminación por bloques`), para ello, en el presente trabajo se implementó un algoritmo directo basado en factorizaciones matriciales (FM): `factorización QR` que permiten encuentrar sistemas de ecuaciones equivalentes (es decir, sistemas de ecuaciones lineales que tengan un mismo conjunto de solución). Una vez calculada la FM de la matriz a resolver, se utilizaron métodos de sustitución hacia adelante y hacia atrás para resolver SEL triangulares.



Para la implementación del presente trabajo, el equipo de programación desarrollo una serie funciones que permiten la implementación del algoritmo antes descrito; dichas funciones se encuentran principalmente en los siguientes el siguiente módulo:

> [funciones_factorizacion_QR.py](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Codigo/funciones_factorizacion_QR.py)

La descripción explícita de las funciones implementadas, los parámetros, resultados esperados y ejemplos de implementación de las funciones, se encuentran en el siguiente documento auxiliar:
> [Help_funciones_factorizacion_QR.txt](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Codigo/Help_funciones_factorizacion_QR.txt)

Durante la implementación de los métodos, gracias a los comentarios y revisiones del equipo revisor, se incluyeron las validaciones correspondientes sobre las condiciones necesarias para la aplicación del método de eliminación por bloques a la matriz $A$: matrix cuadrada ($nxn$) no singular. En caso de que no se cumplieran estas condiciones, el programa implementado arroja mensajes de error, como es esperado. Asimismo, se realizaron las mediciones de tiempo y errores relativos correspondientes para evaluar el desempeño del programa, ya sea por subdivisión de la matriz del sistema o no. 

El objeto del presente notebook es justamente reportar los hallazgos que obtuvimos durante la implementación del programa, ya sea durante de la programación del código o durante la revisión del algoritmo propuesto. Los detalles relacionados a la implementación del framwork de $scrum$ o la organización del trabajo son cubiertos en el [README.md](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/README.md) del repositorio.

## Sobre los métodos empleados

> Algoritmos

### Factorización QR

Existen una gran cantidad de métodos para resolver SEL y por lo general se elige el método de acuerdo con los coeficientes de la matriz del sistema que determinan propiedades de la misma y sus dimensiones. Por lo que, para SEL no triangulares, más generales, es decir, que no cuentan con una estructura identificable, se tienen los métodos iterativos y directos o basados en factorizaciones matriciales (FM). Entre los directos o basados en FM se encuentra la factorización $QR$. 
    
La factorización QR se calculó con reflexiones de Householder, la cual consiste en utilizar matrices simétricas, ortogonales. La factorización $QR$ con refleciones de Householder se puede construir a partir de un vector ($v$), donde $v\in \mathbb{R}^{m}$ y debe ser diferente de cero. Una matrix $R$ de $m$x$m$ de la forma: $$R=I-\beta v v^{T}, \ \ \ \beta=\frac{{2}}{v^{T}v},$$ es una reflexión Householder. El vector $v$ es el vector Householder. Si un vector $x$ se multiplica por una matrix $R$, esto representa la reflexión del vector $x \in \mathbb{R}^{m}$ a través del hiperplano $span\{v\} ^{\perp}$. 

Las reflecciones de Householder son similares a las transformaciones de Gauss, ya que son modificaciones en el rango 1 de la identidad, en particular suponemos que $0 \neq x \in \mathbb{R}^{m}$ y queremos: $$Rx=\left(I - \frac{2vv^{T}}{v^{T}v}\right)x = x - \frac{2v^{T}x}{v^{T}v}v$$ debe ser un multiplo de $e_1=I_m(:,1)$. De esto concluimos que $v \in span\{x,e_1\}$. Ajustando: $$v = x + \alpha e_1$$ da $$v^{T}x = x^{T}x + \alpha x_1$$ y $$v^{T}v = x^{T}.$$ Por tanto, $$Rx = \left(1-2\frac{x^Tx+\alpha x_1}{x^Tx+2\alpha x_1 + \alpha_2}\right)x - 2 \alpha \frac{v^Tx}{v^Tv}e_1.$$ Para que el coeficiente de $x = 0$, ajustamos $\alpha = \pm ||x||_2$, para ello: $$v = x \pm ||x||_2 e_1 \Rightarrow Rx=\left(I - \frac{2vv^{T}}{v^{T}v}\right)x = \mp ||x||_2 e_1.$$ Es esta simple determinación de $v$ que hace que las reflexiones de Householder sean tan útiles.

Ajustando la elección del signo en la definición de $v$ como sigue:$$v_1 = x_1 - ||x||_2,$$ se prueba que $Rx$ es un múltiplo positivo de $e_1$. Pero se debe tener cuidado si $x$ se encuentra cerca de un múltiplo de $e_1$ porque ocurriría una canselación severa. Sin embargo, la fórmula $$v_1=x_1-||x||_2 = \frac{x_1^2-||x||_2^2}{x_1+||x||_2} = -\frac{x_2^2+x_3^2+\dots + x_m^2}{x_1+||x||_2}$$ no sufre esto en el caso de $x_1>0$. En la implementación del cálculo del vector de Householder, es útil que $v_1=1$ y así únicamente se almacenará $v(2:m)$. Al vector $v(2:m)$ se le nombra parte esencial del vector de Householder.
    
Recordando que $\beta = \frac{{2}}{v^{T}v}$, especificando la dimensión del vector $x$ y dado que $x \in \mathbb{R}^{m}$, el algoritmo que se uso para calcular el vector Householder con salida $v \in \mathbb{R}^m$ con $v_1 =1$ y $\beta \in \mathbb{R}$ tal que $R=I_m-\beta vv^T$ es ortogonal con $Rx = ||x||_2e_1$ es:

* función [$v, \beta$] = $house(x)$
* $m =$ longitud de $x$
* $norm\text{_2_m} = x(2:m)^Tx(2:m)$
*$v = \left[\begin{array}{c}
1\\
x(2:m)
\end{array}
\right]$

* Se verifica si $x$ es un múltiplo del vector canónico $e_1$ y el $signo(x_1)$ con las siguientes condiciones:
    
    * Si $norm\text{_2_m} = 0$ y $x_1 \geq 0$, entonces hacer $\beta=0$
    * Si $norm\text{_2_m} = 0$ y $x_1 <0$, entonces hacer $\beta=2$
    * En otro caso: 
        * $norm\text{_x} = \sqrt{x_1^2 + norm\text{_2_m}}$ 
        * Si $x_1 \leq 0$, entonces hacer $v_1 = x_1-norm\text{_x}$
        * En otro caso:
          $v_1 = - \frac{norm\text{_2_m}}{x_1+norm\text{_x}}$
        * Definir: $\beta = \frac{2v_1^2}{norm\text{_2_m}+v_1^2}$,
        * Definir: $v = \frac{v}{v_1}$

En una situación típica house es aplicada a una subsolumna o subfila de una matriz y ($I-\beta v v^{T}$) es aplicado a una submatriz. Por ejemplo, si $A \in \mathbb{R}^{mxn}$, $1 \leq j < n$ y $A(j:m, 1: j-1)$ es cero, entonces  

* [$v, \beta$] = $house((A(j:m,j))$
* $A(j:m, j:n) = A(j:m,j:n)-(\beta v)(v^TA(j:m,j:n))$
* $A(j+1:m,j) = v(2:m-j+1)$

se aplica ($I_{m-j+1}-\beta v v^{T}$) a $A(j:m,1:n)$ y almacena la parte esencial de $v$ donde los "nuevos" ceros son introducidos.

El almacenamiento de los vectores $v^{(1)}, v^{(2)}, \dots, v^{(n)}$ y las $\beta_j$'s correspondientes se le nombra **factored-form representation** de $Q$.    

Muchos algoritmos basados en la factorización de Householder tratan de realizar productos de matrices de Householder, es decir, típicamente no es necesario construir a la matriz $Q$ en la factorización $QR$. Para realizar la multiplicación $Q^TC$ con $C \in \mathbb{R}^{m \times p}$ se realiza lo siguiente:

* Para $j=1:n$
   * $v(j:m) = \left[\begin{array}{c}
1\\
A(j+1:m,j)
\end{array}
\right]$

   * $\beta_j = \frac{2}{1+||A(j+1:m,j)||_2^2}$
   * $C(j:m,:) = C(j:m,:) - (\beta_jv(j:m))(v(j:m)^TC(j:m,:))$

La ventaja del **factored-form representation** es evidente si $n<<m$.

Dada una matriz $A \in \mathbb{R}^{mxn}$ con $m\geq n$, el siguiente algoritmo encuentra matrices Householder $H_1,...,H_n$, tal que si $Q=H_1,...,H_n$ entonces $Q^TA=R$ es triangular superior. La parte triangular de la matriz $A$ se sobrescribe con la parte triangular superior de $R$ y las componentes $j+1:m$ del $j$-ésimo vector de Householder se almacenan en $A(j+1:m,j)$ para $j<m$. Si $v^{(j)} = [0, \dots ,0,1,v_{j+1}^{(j)}, \dots,v_m^{(j)}]^T$ es el $j$-ésimo vector de Householder para $j=1,\dots,m$, por lo que el algoritmo para calcular la factorización QR con reflexiones de Householder es el siguiente:
    
* Para $j =1:n$
    * [$v,\beta$] = $house((A(j:m,j))$
    * $A(j:m, j:n) = A(j:m,j:n)-(\beta v)(v^TA(j:m,j:n))$
    * Si $j<m$
      * $A(j+1:m,j) = v(2:m-j+1)$.
    
Por ejemplo utilizando el algoritmo anterior con $m=4$ y $n=4$ se obtiene lo siguiente:    
        
$$A=\begin{array}{l}
\left[ \begin{array}{cccc}
r_{11} & r_{12} & r_{13} & r_{14}\\
v_2^{(1)} & r_{22} & r_{23} & r_{24}\\
v_3^{(1)} & v_3^{(2)} & r_{33} & r_{34}\\
v_4^{(1)} & v_4^{(2)} & v_4^{(3)} & r_{44}\\
\end{array}
\right]
\end{array}
$$

Donde los valores de las $\beta's$ para cada vector Householder se pueden calcular con la expresión: $$\beta_j = \frac{2}{1+||A(j+1:m,j)||_2^2}$$ para $j=1,\dots,m$.

El algoritmo para la factorización QR esta aplicando transformaciones de Householder a la matriz $A$ por la izquierda. Si $Q_j = I_m - \beta_j v^{(j)}v^{T(j)}$ es la $j$-ésima transformación de Householder y $v^{(j)}$ es el $j$-ésimo vector de Householder, entonces en el ciclo for del algoritmo se realiza el producto $Q^TA$ con $Q=Q_1Q_2 \cdots Q_n$. En cada paso del ciclo for se hacen ceros por debajo de una entrada de la matriz. Por lo tanto es equivalente bajo transformaciones a los reflectores de Householder la matriz A.

### Eliminación por Bloque

En términos generales, el método de eliminación por bloque consiste en encontrar un sistema de ecuaciones equivalentes; para ello elimina un subconjunto de variables. Considere el siguiente sistema de ecuaciones lineales:
$$\left[\begin{array}{c}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{array}
\right] \times \left[\begin{array}{c}
x_1\\
x_2
\end{array}
\right] =\left[\begin{array}{c}
b_1\\
b_2
\end{array}
\right]$$

donde $x_1 \in R^{n_1}, x_2 \in R^{n_2}, A_{11} \in R^{n_1xn_1}, A_{22} \in R^{n_2xn_2}, b_{1} \in R^{n_1}, b_{2} \in R^{n_2}.$ Por lo que, si $A$ es invertible, entonces se puede obtener dos ecuaciones equivalentes al sistema original calculando el complemento de Schur ($S= A_{22}-A_{21}A_{11}^{-1}A_{12}$) del bloque $A_{11}$. 

De la primera ecuación por bloques se obtiene lo siguiente: $$x_{1}= A_{11}^{-1}(b_{1}-A_{12}x_{2},$$ sustituyendo esta relación en la segunda ecuación por bloques se obtiene la ecuación reducida: $$(A_{22}-A_{21}A_{11}^{-1}A_{12})x_{2}=b_{2}-A_{21}A_{11}^{-1}b_{1}$$  

De esta, resolvemos $Sx_2= \hat{b}$ y $A_{11}x_{1}=b_1-A_{12}x_{2}$, con el empleo de factorización QR. 

## Funciones y módulos creados

Todas las función creadas para el proyecto se encuentran dentro del módulo *funciones_factorizacion_QR*. A continuación se enlistan y se da una breve descripción de las funcioines creadas para este proyecto:

- *crear_matriz_aleatoria*: crea matrices aleatorias. Devuelve una matriz $A$ con con los renglones y columnas deseados.
- *house*: calcula el vector de householder para un vector $\vec{v}$ dado.
- *matriz_auxiliar_Arv*: dada una matriz $A$, genera una matriz auxiliar que contiene los elementos $r$ distintos de cero de la matriz $R$ y las entradas de los vectores householder $\vec{v}$ (excepto la primera entrada), con los cuales se puede calcular la matriz $Q$ correspondiente.
- *Q_j*: calcula la j-esima iteración del proceso para la factorización $QR$, regresando la matriz $Q_j$ correspondiente con ayuda de la *matriz_auxiliar_Arv*.
- *matriz_Q_R*: dada una matriz $A$, devuelve la matrices $Q$ y $R$ de la factorización $QR$ apoyándose de la función *Q_j* mediante la aplicación del método iterativo, en el que $Q=Q_1Q_2\ldots Q_n$
- *Solucion_SEL_QR_nxn*: obtiene la solución de un sistema de ecuaciones lineales $Ax=b$ con $n$ ecuaciones y $n$ incognitas usando la factorización $QR$ y usando el método de sustitución hacia adelante del paquete *scipy*.
- *crear_bloques*: dada una matriz $A$, devuelve cuatro bloques que componen a esta matriz, en caso de no dar los argumentos para el tamaño de los bloques devuelve los bloques del mismo tamaño.
- *eliminacion_bloques*: implementa el método de eliminación por bloques para resolver un sistemade ecuaciones usando la factorización $QR$.

El módulo *funciones_factorización_QR* se encuentra en la siguiente liga [funciones](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Codigo/funciones_factorizacion_QR.py)

## Ejemplos de Aplicación

Con la intención de mostrar la funcionalidad de los algoritmos implementados por el equipo de programación, el equipo de revisión llevó a cabo una serie de mediciones clasificadas en los siguiente tres rubros:
+ Medición de tiempos a nivel granular
+ Medición de tiempos y errores a nivel general
+ Medición de tiempos y errores variando el tamaño de bloques

El desarrollo explícito de estas mediciones, puede encontrarse en la siguiente liga [insumos](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/tree/master/Reporte/Insumos).

En seguida se presentarán los resultados relacionados con cada una de las mediciones.


>Medición de tiempos a nivel granular

La función general de este proyecto, **eliminacion_bloques** utiliza a su vez un conjunto de funciones para resolver los sistemas de ecuaciones lineales. Estas funciones de apoyo pueden enunciarse particularmente conforme a lo siguiente:

1. matriz_auxiliar_Arv
2. matriz_Q_R (se hace uso de la función matri_auxiliar_Arv)
3. crear_bloques
4. Solucion_SEL_QR_nxn (se hace uso de la función matriz_Q_R)

Debido a la presencia de estas cuatro ecuaciones, se decidió medir sus tiempos de ejecución para distintos tamaños de matrices. Adicionalmente se incluyó también la medición de la matriz que permite crear matrices aleatorias (*crear_matriz_aleatoria*), si bien esta función no forma parte de la función general *eliminacion_bloques*, su uso es determinante para la creación de los sistemas de ecuaciones.

Es importante mencionar que la función *Solucion_SEL_QR_nxn* es utilizada cuatro veces en el algoritmo *eliminacion_bloques*; sin embargo, para este caso se midieron únicamente los tiempos asociados a la solución del sistema de ecuaciones correspondiente al primer bloque, es decir, $A_{11}y = b_1$

La medición de tiempos de ejecución se llevó a cabo sobre cuatro matrices de dimensiones $20x20$, $10x10$, $100x100$ y $500x500$; y los detalles relacionados con las mismas se muestran en [Tiempos_granular](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Reporte/Insumos/Tiempos_granular.ipynb). 

Como resumen de los resultados obtenidos, destacan los siguientes puntos:
+ En general, los tiempos de ejecución de las funciones aumentan conforme aumentan las dimensiones de la matriz. Excepto en la función *crear_bloques*, la cual mantiene un tiempo constante en la creación de submatrices para las 4 matrices que fueron probadas.
+ El hecho de que la función *crear_bloques* mantenga un tiempo constante puede deberse a que no se implementa ningún ciclo for, a diferencia del resto de las funciones, en donde al menos un ciclo es incluido.
+ La función matriz_QR es la que tiene asociada los tiempos más altos de ejecución, alcanzando los $3s$ en matrices de tamaño $500x500$.
+ La función *matriz_auxiliar_Arv* y *Solucion_SEL_QR* son las que presentan menores tiempos de ejecución, variando entre $10^{-4} s$ y $10^{-1} s$.

>Medición de tiempos y errores a nivel general

En esta sección mostraremos los tiempos de ejecución y los errores relativos para matrices con dimensiones 2x2, 10x10, 100x100 y 500x500  que obtuvimos para la función **eliminacion_bloques**. Cabe mencionar que esta función engloba a todas las funciones de la sección anterior. 

Para la parte de errores relativos, estaremos comparando los resultads de la función implementada por los programadores contra la librería numpy.

Observemos la siguiente tabla:

<p align="center">
  <img width="460" height="300" src="Images/Tabla_tiempos_errores_general.png">
 
 **Tabla**: Tiempos de ejecución y promedio de errores relativos 
   
</p>

Se puede identificar que al aumentar las dimensiones de las matrices de entrada, tanto los tiempos de ejecución como el promedio del error relativo van aumentando. 

Particularmente se aprecia un crecimiento muy marcado en los tiempos de ejecución cuando cambiamos de una matriz 100x100 a una de 500x500. Mientras que por otro lado, el cambio en el promedio del error relativo es mucho mas sutil para este mismo caso.



<p align="center">
  <img width="460" height="300" src="Images/Graf_tiempos_ejecucion.png">
 
 **Gráfica**: Tiempo de ejecución
   
</p>

Obsérvese el cambio en el tiempo de ejecución que mencionabamos anteriormente al cambiar de una matriz de 100x100 a 500x500.


<p align="center">
  <img width="460" height="300" src="Images/Graf_errores.png">
 
 **Gráfica**: Error relativo
   
</p>

Podemos observar que tambien existe un crecimiento importante en el promedio del error relativo al cambiar de una matriz de 100x100 a 500x500, sin embargo la pendiente de esta gráfica es menos empinada que la de los tiempos de ejecución para este mismo caso.

[Referencia al notebook](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Reporte/Insumos/Tiempos_Errores_General.ipynb)

>Medición de tiempos y errores variando el tamaño de bloques

La función general de este proyecto, **eliminacion_bloques** permite elegir los tamaños de bloques para resolver ecuaciones lineales.

Se decidió meditr el tiempo de ejecución y el error relativo para distintos tamaños de bloques. Los tamaño de bloques que se utilizaron son: 20x20, 40x40, 60x60 y 80x80. La tabla y graficas siguientes muestran los resultados 

<p align="center">
  <img width="460" height="300" src="Imagenes/tabla_distintos_tamanios_bloques.PNG">

**Tabla**: Tiempos de ejecución y promedio de errores relativos 
   
</p>


<p align="center">
  <img width="460" height="300" src="grafica_tiempo_distintos_tamanios_bloques.PNG">
    
**Gráfica**: Tiempo de ejecución
   
</p>    

<p align="center">
  <img width="460" height="300" src="grafica_error_distintos_tamanios_bloques.PNG">

 **Gráfica**: Error relativo
   
</p>

Observamos que el menor tiempo de ejecución se presenta cuando elegimos bloques más pequeños, mientras que el menor error relativo es cuando elegimos bloques de mayor tamaño.

## Pruebas

La forma en que el equipo de revisión validó las funciones implementadas por el equipo de programación fue a través de dos conjuntos de pruebas. En primer lugar, la Revisión 1 tuvo como objetivo identificar posibles errores en la primera versión de los algoritmos. Por su parte, la segunda revisón tuvo la intención de validar que el equipo de programación hubiera atendido las observaciones hechas durante la Revisión 1 y, adicionalmente, verificar que todas las funciones estuvieran concentradas en un único módulo: *funciones_factorizacion_QR.py*. Los detalles de cada prueba están ubicados en [Pruebas](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/tree/master/Pruebas).

Este conjunto de pruebas fue dividido en cuatro secciones:
+ Factorización QR
+ Sistemas con solución única
+ Sistemas con soluciones infinitas
+ Sistemas con infinitas soluciones



>Factorización QR

>Sistemas con solución única

>Sistemas con infinitas soluciones

Dentro de este tipo de pruebas se incluyeron matrices de entrada con soluciones infinitas de dimensiones $2x2$ y $10^{2}x10^{2}$.

[Revisión 1](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Pruebas/Revision1/pu_epbQR_sistema_soluciones_infinitas.ipynb) En la primera revisión, la función *eliminacion_bloques* marcaba un error al momento de incluir los sistemas con soluciones infinitas, el cual no era claro para el usuario, el detalle específico puede consultarse en la referencia Revisión1. 

[Revisión 2](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Pruebas/Revision2/pu2_epbQR_sistema_soluciones_infinitas.ipynb) Para la segunda revisión, el equipo de programación incluyó la leyenda: "A debe ser no singular", el cual se despliega al momento de ingresar un sistema de ecuaciones con soluciones infinitas. De cara al usuario que está ejecutando la función, ésto resulta muy útil para entender de forma muy directa el problema con el que se está topando el algoritmo.

Por otro lado, en los ejemplos 2 y 3 [soluciones infinitas](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Pruebas/Revision2/pu2_epbQR_sistema_soluciones_infinitas.ipynb) se mantuvo el mismo lado izquierdo $A$ y se modificó el lado derecho $b$, de forma que el Ejemplo 3 es un sistema homogéneo y el Ejemplo 2 no lo es. En ambos casos el error asociado indica que la matriz $A$ deber ser no singular. Por lo tanto la solución del sistema de ecuaciones depende en su totalidad del lado izquierdo $A$.


>Sistemas sin solución

[Revisión 1](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Pruebas/Revision1/pu_epbQR_sistema_sin_solucion.ipynb) Durante esta primera revisión nos dimos cuenta que la función **eliminacion_bloques** no era capaz de identificar cuando se enfrentaba a un sistema de ecuaciones lineales sin solución ni cuando se le daba como insumo una matriz no cuadrada. Se probaron varios casos para corroborar lo anterior:
+ Matriz con fila de ceros
+ Sistema incompatible - Rectas Paralelas
+ Matriz no cuadrada

Se levantarón los respectivos issues para agregar esta funcionalidad, con la finalidad de retroalimentar al usuario cuando se llegara a enfrentar a este tipo errores al utilizar la función.

[Revision 2](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Pruebas/Revision2/pu2_epbQR_sistema_sin_solucion.ipynb) En esta segunda revisión validamos que los issues levantados en la revisión previa fueran atendidos satisfactoriamente por el equipo de programación, lo cual se cumplió en tiempo y forma.

## Conclusiones

## Referencias